# I am confused which dataset we are supposed to use, the assignment says 5core... we need to adjust the defenitions below and add also the correct file we will use.

the project is almost donne, we just have to do the due dilligience and develop the content filtering text based

# RECOMMENDATION ENGINES - AMAZON TOYS AND GAMES

## GROUP C
- Nikolas Artadi
- Camila Vasquez
- Assemgul Khametova
- Miguel Frutos

## TASK
- **DATA SELECTION AND PRE-PROCESSING**(Mandatory)
First, you need to select a product category (from the “Small subsets for experiment”) and download the related file to create a training dataset and a testing dataset for the experiment. A recommended standard pre-processing strategy is that: each user randomly selects 80% of their ratings as the training ratings and uses the remaining 20% ratings as testing ratings.
- **COLLABORATIVE FILTERING RECOMMENDER SYSTEM** (Mandatory)
Based on the training dataset, you should develop a Collaborative Filtering model/algorithm to predict the ratings in the testing set. You may use any existing algorithm implemented in Surprise (or any other library) or develop new algorithms yourself. After predicting the ratings in the testing set, evaluate your predictions by calculating the RMSE.
- **CONTENT-BASED RECOMMENDER SYSTEM** (Mandatory)You should leverage the textual
information related to the reviews to create a Content-based RS to predict the ratings for the users in the test set. I do recommend you make use of the lab session related to the topic.
- **HYBRID HS**(Optional)
As an extra, you can propose a hybrid recommender system joining the operation of the two previously developed systems. To that end, you can make use of any of the ideas explained in class.
    
## DATASET
We follow the data set Toys and Games in the following [source](http://deepyeti.ucsd.edu/jianmo/amazon/index.html).

### Ratings only features explanation
- **reviewerID** - ID of the reviewer, e.g. A2SUAM1J3GNN3B
- **asin** - ID of the product, e.g. 0000013714
- **reviewerName** - name of the reviewer
- **helpful** - helpfulness rating of the review, e.g. 2/3
- **reviewText** - text of the review
- **overall** - rating of the product
- **summary** - summary of the review
- **unixReviewTime** - time of the review (unix time)
- **reviewTime** - time of the review (raw)

### Metadata
- **asin** - ID of the product, e.g. 0000031852
- **title** - name of the product
- **price** - price in US dollars (at time of crawl)
- **imUrl** - url of the product image
- **related** - related products (also bought, also viewed, bought together, buy after viewing)
- **salesRank** - sales rank information
- **brand** - brand name
- **categories** - list of categories the product belongs to

Where did you get this information?
> ### FEATURES
- **user-id**: which is denoted as “reviewerID” in the dataset
- **product-id**: which is denoted as “asin” in the dataset
- **rating**: a 1-5 integer star rating, which is the rating that the user rated on the product, it is denoted as “overall” in the dataset
- **review**: a piece of review text, which is the review content that the user commented about the product, it is denoted as “reviewText” in the dataset
- **title**: the title of the review, which is denoted as “summary” in the dataset
- **timestamp**: time that the user made the rating and review
- **helpfulness**: contains two numbers, i.e., [#users that think this review is not helpful,
#users that think this review is helpful]

# we still need to create e training dataset... we can do this once we decide what other metadata we want to use for the content based and hybrid... we will need to merge the data set here....

# LET´S GET STARTED

 ## LIBRARIES INSTALATION

In [1]:
# ! pip install scikit-surprise
# ! pip install plotly
# ! pip install seaborn

import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split

from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise import SVDpp
from surprise import SVD
from surprise import KNNBaseline
from surprise import SVDpp
from surprise import NormalPredictor
from surprise import BaselineOnly
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering

from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

import matplotlib.pyplot as plt

## READ DATA

In [4]:
# df = pd.read_json('game_toy.json',lines=True)

# Ratings data
df = pd.read_json('game_toy.json',lines=True)

# Split dataset into training (df) and test dataset at 80% for training of the total data
df, test_dataset = train_test_split(df, train_size=0.80)

## ANALYZE THE DATA

Take a quick look at the data to check if the dataset is correctly uploaded and to understand the variable´s content and the schema.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134077 entries, 160029 to 3271
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   reviewerID      134077 non-null  object
 1   asin            134077 non-null  object
 2   reviewerName    133414 non-null  object
 3   helpful         134077 non-null  object
 4   reviewText      134077 non-null  object
 5   overall         134077 non-null  int64 
 6   summary         134077 non-null  object
 7   unixReviewTime  134077 non-null  int64 
 8   reviewTime      134077 non-null  object
dtypes: int64(2), object(7)
memory usage: 10.2+ MB


In [5]:
df.describe()
# We can see that the ratings have min of 1 and max of 5

,overall,unixReviewTime
count,134077.000000,1.340770e+05
mean,4.359197,1.348601e+09
std,0.990976,6.099707e+07
min,1.000000,9.662976e+08
25%,4.000000,1.335658e+09
50%,5.000000,1.364256e+09
75%,5.000000,1.388016e+09
max,5.000000,1.406074e+09


In [6]:
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
92893,A2I3TKKHF6INI6,B004KZ8P2Q,Connie,"[0, 0]",I've been waiting for my little boy (born Janu...,4,A good first board game for 3 year olds,1342656000,"07 19, 2012"
166772,A2Z780XULPMGZ8,B00H655PM0,Loyal1369,"[0, 0]",Gotta love this version of the joker. The fact...,5,Why so serious,1397692800,"04 17, 2014"
18601,A2ICCKJYD394HT,B000284X6W,"Amanda ""AStallings""","[2, 2]",My daughter has everything in the laugh and le...,5,Love the Laugh and Learn Line!,1152144000,"07 6, 2006"
117345,A36AQ69ZYQAKCU,B006WIQMRE,globaltrekkie,"[5, 5]","Great item for any Star Wars fan, young or old...",5,Great item,1345939200,"08 26, 2012"
106710,A2HELIKP5RV27F,B005G2R3K2,"L. Marrott ""Books&Amazon4Me""","[1, 2]","If you enjoy Monopoly, this is a fun new twist...",4,Good New Spin on a Classic,1316044800,"09 15, 2011"
...,...,...,...,...,...,...,...,...,...
94553,A3LF53L0WE6CWM,B004NWOHLE,momofsevencuties,"[0, 0]",my daughter loves this pillow. it is super cut...,5,precious,1392076800,"02 11, 2014"
158724,A393AF9SFW08FA,B00CYQ24QG,Leah S Bush,"[0, 0]",My kids played this at a friend's house and be...,3,Okay but not super great.,1396656000,"04 5, 2014"
161876,A2GR8DKPGFS0FX,B00E7TKSWK,Maga loves her grand kids !,"[0, 1]",Everyone has a rainbow loam and they weave awa...,5,My granddaughters loved them,1388966400,"01 6, 2014"
76045,A263ZEDYD50VAT,B003A0GTN4,sbmiller,"[0, 0]",I am so in love with all the Mellisa and Doug ...,5,perfect!,1302825600,"04 15, 2011"


We have included an EDA and have identified duplicates and missing data which we will handle in the next steps.

In [7]:
def missing_values_percentage(df):
    """Return the % of missing values for each pd.series inside the Dataframe"""
    for i in df:
        missing_values_percentage = 100*df.isnull().sum()/df.isnull().count()
    return (missing_values_percentage[missing_values_percentage > 0]) 

In [8]:
missing_values_percentage(df)

reviewerName    0.488525
dtype: float64

In [9]:
#Decided to drop column review name as we have a 50% of missing values and the adding value of this column is zero.
del df['reviewerName']

In [10]:
missing_values_percentage(df)

Series([], dtype: float64)

In [11]:
#Drop duplicates
df.drop_duplicates
#Result, zero entire duplicated rows in game_toy dataset

<bound method DataFrame.drop_duplicates of             reviewerID        asin   helpful  \
92893   A2I3TKKHF6INI6  B004KZ8P2Q    [0, 0]   
166772  A2Z780XULPMGZ8  B00H655PM0    [0, 0]   
18601   A2ICCKJYD394HT  B000284X6W    [2, 2]   
117345  A36AQ69ZYQAKCU  B006WIQMRE    [5, 5]   
106710  A2HELIKP5RV27F  B005G2R3K2    [1, 2]   
...                ...         ...       ...   
94553   A3LF53L0WE6CWM  B004NWOHLE    [0, 0]   
158724  A393AF9SFW08FA  B00CYQ24QG    [0, 0]   
161876  A2GR8DKPGFS0FX  B00E7TKSWK    [0, 1]   
76045   A263ZEDYD50VAT  B003A0GTN4    [0, 0]   
4479    A2SHQJP6PNQTLD  B00000IWIA  [74, 83]   

                                               reviewText  overall  \
92893   I've been waiting for my little boy (born Janu...        4   
166772  Gotta love this version of the joker. The fact...        5   
18601   My daughter has everything in the laugh and le...        5   
117345  Great item for any Star Wars fan, young or old...        5   
106710  If you enjoy Monopoly,

# this is not correct, I have added something here.. not sure what the column means..

In [12]:
df.sort_values("helpful", ascending=False).head(5)

,reviewerID,asin,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
46315,A1OUQCTNVKPVR9,B0010VS078,"[1589, 1637]",I loaned my iPod to my kid and he broke it. T...,4,It's a great portable music solution,1270166400,"04 2, 2010"
103098,A4LD7XC56J3ZV,B004Z7H07K,"[1431, 1502]",Hi! I am Erin T. and I run a website called th...,5,My Son Won't Put it Down,1313712000,"08 19, 2011"
131030,A1SC7Z2646QCP9,B0089RPUHO,"[1413, 1449]",If you want a child-friendly tablet-style devi...,5,Hands down the best choice for a child-friendl...,1350864000,"10 22, 2012"
80422,A3DZFEICHK5LF2,B003JQT4Y0,"[1378, 1393]","Short version:The good: The pen is amazing, a ...",3,Great product but a lot more parent involvement.,1285632000,"09 28, 2010"
103019,A2DG63DN704LOI,B004Z7H07K,"[1291, 1359]",I really want to like the LeapPad - my kids do...,3,"Kids like it, but educational value is not as ...",1315612800,"09 10, 2011"


In [13]:
df['users_nothelpful']=df.helpful.str[0]
df['users_helpful']=df.helpful.str[1]

# Analyze the data

### See the count of ratings per rating

In [14]:
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

# Count the number of times each rating appears in the dataset
data = df['overall'].value_counts().sort_index(ascending=False)

# Create the histogram
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / df.shape[0] * 100)],
               textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,
               )
# Create layout
layout = dict(title = 'Distribution Of {} Toys and Games Ratings'.format(df.shape[0]),
              xaxis = dict(title = 'Rating'),
              yaxis = dict(title = 'Count'))
# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

Most of the reviews are at 5, this shows an extremely skewed graph and therefore we can infer that the ratings given in this data set are strongly biased as most lay between 4 and 5. We have over 80% of users rating positively the product.

### See the number ratings per product

In [15]:
# Number of ratings per game_toy
data = df.groupby('asin')['overall'].count()

# Create trace
trace = go.Histogram(x = data.values,
                     name = 'overall',
                     xbins = dict(start = 0,size = 2))
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per Product',
                   xaxis = dict(title = 'Number of Ratings Per Product ID'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

We can clearly see a long tail of reviews per product. Only almost 50 products have most of the total reviews and the rest have extremely low reviews.

### See the number of ratings per users

In [16]:
# Number of ratings per user
data = df.groupby('reviewerID')['overall'].count()
# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0, size = 2))
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per User',
                   xaxis = dict(title = 'Ratings Per User'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In this segement, the similar scenario happened as in the previous case. We see a high distribution among a small amount of users and an extremely long tail among the latter ones.

We can now savely remove those users that fall below our threshhold of at least 50 reviews.

In [17]:
# Removing those users that have a rating below the threshhold

tresh = 50
sub_df = df[df.groupby('reviewerID')['overall'].transform('count')>tresh].copy() 
print('Old shape: ',df.shape[0],'rows')
print('New shape: ',sub_df.shape[0],'rows')
print('Difference: ',-df.shape[0]+sub_df.shape[0],'rows')

Old shape:  134077 rows
New shape:  4846 rows
Difference:  -129231 rows


# Start of the collaborative filtering RS

In [20]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['reviewerID', 'asin', 'overall']], reader)

### 1. KNN

In [21]:
# # To use item-based cosine similarity
# sim_options = {
#     "name": "cosine",
#     "user_based": False,  # Compute  similarities between items
# }
# knn = KNNBaseline(sim_options=sim_options)

In [22]:
sim_options = {'name':'pearson_baseline'}

knn = KNNBaseline(k=40,min_k=2,sim_options=sim_options,verbose=True)

results = cross_validate(knn,data,measures=['RMSE','MAE'],cv=5,verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9037  0.9119  0.9053  0.9155  0.9126  0.9098  0.0045  
MAE (testset)     0.6849  0.6908  0.6855  0.6920  0.6899  0.6886  0.0029  
Fit time          16.21   20.06   19.14   19.16   18.33   18.58   1.31    
Test time         1.00    1.04    0.74    0.82    0.7

The RMSE is almost 91 with slights variations. Which shows a stable results over the 5 folds.

In [23]:
cross_validate(NormalPredictor(), data, measures=['RMSE'], cv=3, verbose=True)

Evaluating RMSE of algorithm NormalPredictor on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.2700  1.2690  1.2717  1.2703  0.0011  
Fit time          0.14    0.17    0.17    0.16    0.01    
Test time         0.71    0.27    0.27    0.42    0.21    


{'test_rmse': array([1.26999663, 1.26903753, 1.27172802]),
 'fit_time': (0.13785815238952637, 0.16961216926574707, 0.16848278045654297),
 'test_time': (0.7132139205932617, 0.2708778381347656, 0.271378755569458)}

We can understand that KNN is learning from the dataset given. At this point almost 1.27 is the threshold for all the whole dataset.

Tuning the KNN

In [24]:
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNBaseline, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matr

The best configuration corresponds to a item-based configuration using Mean Square distance with a min support equals to 5, which is able to slightly reduce the RMSE to 0.92. Which is similar to the first baseline we ran with KNN.

### 2. Matrix Factorization

In [25]:
from surprise import SVDpp

# We'll use the famous SVD algorithm.
svd = SVDpp()

results = cross_validate(svd, data, measures=['RMSE'], cv=3, verbose=False)

### 3. Benchmarking

In [26]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), BaselineOnly(), CoClustering()]:
    
    print("Testing {}".format(algorithm))
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')    

Testing <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7ff28511fb50>
Testing <surprise.prediction_algorithms.matrix_factorization.SVDpp object at 0x7ff286d44e20>
Testing <surprise.prediction_algorithms.slope_one.SlopeOne object at 0x7ff2b8b0b940>
Testing <surprise.prediction_algorithms.matrix_factorization.NMF object at 0x7ff2b7f2cd90>
Testing <surprise.prediction_algorithms.random_pred.NormalPredictor object at 0x7ff2b7f2ca00>
Testing <surprise.prediction_algorithms.knns.KNNBaseline object at 0x7ff2aae4a730>
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Testing <surprise.prediction_algorithms.baseline_only.BaselineOnly object at 0x7ff2b7e119a0>
Estimating biases using als...
Estimating biases using als

,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,0.907011,16.966065,1.038884
SVD,0.913653,5.896014,0.431504
BaselineOnly,0.915449,0.417290,0.344096
KNNBaseline,0.967368,14.069173,1.616412
CoClustering,1.015816,3.649199,0.229103
SlopeOne,1.067399,3.652080,0.610951
NMF,1.136230,8.819822,0.480543
NormalPredictor,1.271858,0.150402,0.399184


SVDpp does take the longest but has the best performance. In this case, there is only a somewhat better performance than SVD and the Baseline and depending on how fast we want customer to be influenced by the recommendation we can select a fast model like SVD or even an easy one like BaselineOnly. It has a fast test time and is not too far away as the rest of the results.

In [30]:
df

,reviewerID,asin,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,users_nothelpful,users_helpful
92893,A2I3TKKHF6INI6,B004KZ8P2Q,"[0, 0]",I've been waiting for my little boy (born Janu...,4,A good first board game for 3 year olds,1342656000,"07 19, 2012",0,0
166772,A2Z780XULPMGZ8,B00H655PM0,"[0, 0]",Gotta love this version of the joker. The fact...,5,Why so serious,1397692800,"04 17, 2014",0,0
18601,A2ICCKJYD394HT,B000284X6W,"[2, 2]",My daughter has everything in the laugh and le...,5,Love the Laugh and Learn Line!,1152144000,"07 6, 2006",2,2
117345,A36AQ69ZYQAKCU,B006WIQMRE,"[5, 5]","Great item for any Star Wars fan, young or old...",5,Great item,1345939200,"08 26, 2012",5,5
106710,A2HELIKP5RV27F,B005G2R3K2,"[1, 2]","If you enjoy Monopoly, this is a fun new twist...",4,Good New Spin on a Classic,1316044800,"09 15, 2011",1,2
...,...,...,...,...,...,...,...,...,...,...
94553,A3LF53L0WE6CWM,B004NWOHLE,"[0, 0]",my daughter loves this pillow. it is super cut...,5,precious,1392076800,"02 11, 2014",0,0
158724,A393AF9SFW08FA,B00CYQ24QG,"[0, 0]",My kids played this at a friend's house and be...,3,Okay but not super great.,1396656000,"04 5, 2014",0,0
161876,A2GR8DKPGFS0FX,B00E7TKSWK,"[0, 1]",Everyone has a rainbow loam and they weave awa...,5,My granddaughters loved them,1388966400,"01 6, 2014",0,1
76045,A263ZEDYD50VAT,B003A0GTN4,"[0, 0]",I am so in love with all the Mellisa and Doug ...,5,perfect!,1302825600,"04 15, 2011",0,0


## Try out the solution

In [31]:
# Execute KNN
sim_options = {'name': 'pearson_baseline', 'user_based': False}
knn = KNNBaseline(sim_options=sim_options)
knn.fit(data.build_full_trainset())

# Target movie to analyze its neighbourhood
game_name = 'B00H655PM0'

# Get the closes neighbourds
neighbors = knn.get_neighbors(knn.trainset.to_inner_iid(game_name), k=10)
# Translate the internal ids used in the algorithm to the movie names
neighbors = (knn.trainset.to_raw_iid(inner_id) for inner_id in neighbors)

print()
print('The 10 nearest neighbors of {} are:\n'.format(game_name))
for game in neighbors:
    print("\t",game)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.

The 10 nearest neighbors of B00H655PM0 are:

	 B00AAOJ142
	 B00H64YYWS
	 B00H64U77Y
	 B00LBI9BKA
	 B009MDU3CG
	 B00727WRQ4
	 B00H7LKW8U
	 B00CPACBQO
	 B00JV9D9M2
	 B004KZ8P2Q


Is not really usefull at first sight, we can only define the product once we get the related product type. Nonetheless, we can pass this to our servers and make a join to define what to recommend to our users.

## Analyze bias

> Best games

In [38]:
game_name = [(b, data.build_full_trainset().to_raw_iid(i)) for i, b in enumerate(svd.bi)]
print("Best games:")
sorted(game_name, key=lambda x: x[0])[:15]

KeyboardInterrupt: 

> Worst games

In [37]:
print("Worst games:")
sorted(game_name, key=lambda x: -x[0])[:15]

Worst games:


NameError: name 'game_name' is not defined

### User bias

In [ ]:
user_bias = [(b, data.build_full_trainset().to_raw_uid(i)) for i, b in enumerate(svd.bu)]
sorted(user_bias, key=lambda x: x[0])[0]

In [ ]:
df[df.userID == XXX]

In [ ]:
sorted(user_bias, key=lambda x: x[0])[-1]

In [ ]:
df[df.userID == XXX]

# CONTENT BASED FILTERING

In [ ]:
import pandas as pd

#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('./ml-latest-small/movies.csv')
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('./ml-latest-small/ratings.csv')
#Head is a function that gets the first N rows of a dataframe. N's default is 5.
movies_df.head()

In [ ]:
#Every genre is separated by a | so we simply have to call the split function on |
movies_df['genres'] = movies_df.genres.str.split('|')

#Copying the movie dataframe into a new one since we won't need to use the genre information in our first case.
moviesWithGenres_df = movies_df.copy()

#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
        
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

In [ ]:
user_id = 2

# Get from the ratings dataframe only the rows (ratings) related to the user_id
user_rating = ratings_df[ratings_df.userId == user_id]
user_rating.drop("timestamp", 1)

# Merge with the movies dataframe to add the movie title to facilitate the analysis of the results
inputMovies = pd.merge(user_rating, movies_df, on='movieId').drop("timestamp",1).drop("genres",1).drop("userId",1)
inputMovies

In [ ]:
#Filtering out the movies from the input
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

In [ ]:
#Resetting the index to avoid future issues
userMovies = userMovies.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1)
userGenreTable

In [ ]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
userProfile

In [ ]:
#Now let's get the genres of every movie in our original dataframe
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
#And drop the unnecessary information
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1)
genreTable.head(10) #This is for all movies

In [ ]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())

#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)

recommendationTable_df

In [ ]:
recommendationTable_df.drop(inputMovies.movieId, inplace=True)

# Content based - Textual Features

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import linear_kernel 
ds = pd.read_csv("sample-data.csv")

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(analyzer='word', min_df=0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(ds['description'])


In [ ]:
feature_names = tfidf.get_feature_names()
df = pd.DataFrame(tfidf_matrix.todense(), columns=feature_names)
df

In [ ]:
dict(df.sort_values(by=1, ascending=False, axis=1).iloc[1])


In [ ]:
from sklearn.metrics.pairwise import linear_kernel 

# Compute cosine similarity
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

# Iterate over the items in the dataset to find the most similar ones to each one
results = {}
for idx, row in ds.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1] 
    similar_items = [(cosine_similarities[idx][i], ds['id'][i]) for i in similar_indices] 
    results[row['id']] = similar_items[1:]

In [ ]:
def item(id):  
    return ds.loc[ds['id'] == id]['description'].tolist()[0].split(' - ')[0] 

# Just reads the results out of the dictionary
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")   
    print("-------")
    recs = results[item_id][:num]   
    for rec in recs: 
        print("Recommended: " + item(rec[1]) + " (score:" +      str(rec[0]) + ")")

In [ ]:
recommend(item_id=11, num=5)

# Conclusions